In [1]:
!pip install openpyxl

In [2]:
!pip install kiwipiepy

     ---------------------------------------- 0.0/79.5 MB ? eta -:--:--
     --------- ----------------------------- 18.9/79.5 MB 92.1 MB/s eta 0:00:01
     ----------------- --------------------- 36.4/79.5 MB 89.2 MB/s eta 0:00:01
     -------------------------- ------------ 53.7/79.5 MB 87.8 MB/s eta 0:00:01
     ------------------------------------- - 76.3/79.5 MB 92.2 MB/s eta 0:00:01
     --------------------------------------- 79.5/79.5 MB 83.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 86.0 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.22.1-py3-none-any.whl size=79578122 sha256=dc6e86896a1c6bec436458bf17f0421623efca68d46194a4cc1152c14e5fa541
  Stored in directory: c:\users\b4-23\appdata\local\pip\cache\wheels\c2\cd\69\f6ec7f41232d2e0aeb74cfc3ee09cb

  DEPRECATION: Building 'kiwipiepy_model' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'kiwipiepy_model'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [3]:
from openpyxl import load_workbook
from kiwipiepy import Kiwi # 대문자면 클래스

src = r"naverkeyword.xlsx" # r = raw 
out = r"naverkeyword_result.xlsx"

wb = load_workbook(src)
ws = wb.active #ws = work sheet, 활성화된 시트를 열겠다

def find_col_by_header(ws, keywords, header_row = 1) : #ws에서 kewords라는 컬럼을 1번째 행에서 찾아오라는 함수 선언
    # keywords = ["연관키워드", "월간검색수 (PC)", "월간검색수 (MOBILE)", .... , "월평균노출 광고수"] -> 리스트
    for col in range(1, ws.max_column + 1) : # 1부터 9까지의 값을 구하려면, (1, 10)의 값을 넣어야 한다
        v = ws.cell(row=header_row, column=col).value # value = 값을 찾아라
        if not v : #만약에 중간에 값이 비어있다면
            continue #아래 명령을 따르지 말고 계속 해라. 비어있지 않다면 아래 명령을 따라라.
        s = str(v).replace(" ", "") 

        ok = True
        for kw in keywords :
            if kw.replace(" ", "") not in s :
                ok = False
                break
        if ok :
            return col 
    return None
    
print(ws)

<Worksheet "sheet">
